In [39]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict

sts = Path.cwd()/'datasets'/'sts'/'sts2016-english-with-gs-v1.0'

sts_ds_all   = list(sts.glob('STS2016.*.txt'))
sts_ds_input = sorted([p for p in sts_ds_all if p.suffixes[0] == '.input'])
sts_ds_gold  = sorted([p for p in sts_ds_all if p.suffixes[0] == '.gs'])
sts_ds_input_gold = list(zip(sts_ds_input, sts_ds_gold))

sts_ds = defaultdict(list)
get_theme = lambda p: p.suffixes[1][1:]

for file_input, file_gold in sts_ds_input_gold:
    theme = get_theme(file_input)
    with open(file_gold) as gold: 
        scores = [np.nan if not line.strip() else int(line.strip()) for line in gold.readlines()]
        sts_ds['score'] += scores
        sts_ds['theme'] += [theme]*len(scores)
        
    with open(file_input) as inp:
        for line in inp:
            if line:
                sent1, sent2 = line.split('\t')[:2]
                sts_ds['sent1'].append(sent1); sts_ds['sent2'].append(sent2)

sts_df = pd.DataFrame(sts_ds)[['sent1', 'sent2', 'theme', 'score']]
sts_df.sort_values(by=['theme', 'score'], ascending=False).to_csv(sts.parent/'STS2016_full.tsv', sep='\t', index=False)